```
Example协议块    
一个example描述一个样本，一个样本有很多features：比如label、image、
message Example {
  Features features = 1;
};
 
message Features {
  map<string, Feature> feature = 1;
};
 
message Feature {
  oneof kind {
    BytesList bytes_list = 1;
    FloatList float_list = 2;
    Int64List int64_list = 3;
  }
```

```
Example协议块    
一个example描述一个样本，一个样本有很多features：比如label、image、
message Example {
  Features features = 1;
};
 
message Features {
  map<string, Feature> feature = 1;
};
 
message Feature {
  oneof kind {
    BytesList bytes_list = 1;
    FloatList float_list = 2;
    Int64List int64_list = 3;
  }
```

#### 1. 将输入转化成TFRecord格式并保存。

In [ ]:
def generate_tfrecords():
    # 创建文件
    writer = tf.io.TFRecordWriter('sequence_example.tfrecord')
 
    # 非序列数据,即每一个元素长短是固定的
    Y = [1, 2, 3, 4, 5, 1, 2, 3, 4]
    # 长度不固定的序列
    X = [[1.0], [2.0, 2.0], [3.0, 3.0, 3.0], [4.0, 4.0, 4.0, 4.0], [5.0, 5.0, 5.0, 5.0, 5.0],
         [1.0], [2.0, 2.0, 3.0], [4.0, 4.0, 4.0, 4.0],[6.0,7.0,8.0]]
 
    for i in tqdm(range(len(Y))):  # 对于每个样本
        y = Y[i]
        x = X[i]
        
        # 第一步：构造各个Feature
        # 非序列化数据，构造一个feature
        y_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[y]))  # 必须包装成list形式
        # 序列化数据，构造一个featurelist，每一个元素是一个Feature,
        x_feature = tf.train.FeatureList(feature= [tf.train.Feature(float_list=tf.train.FloatList(value = [x_])) for x_ in x])  # x_必须包装成列表的形式
    
        # 第二步：构造Features和FeatureLists
        features = tf.train.Features(feature={"Y": y_feature})
        featurelists = tf.train.FeatureLists(feature_list={"X": x_feature})
        
        # 第三步：构造一个Example
        # 构造一个example，来存放一组样本
        sequence_example = tf.train.SequenceExample(
            # context 来放置非序列化部分
            context=features,
            # feature_lists 放置变长序列
            feature_lists=featurelists
        )
 
        serialized = sequence_example.SerializeToString()  # 对没一个样本进行序列化，所以实际上每一个元素就是serialized
        writer.write(serialized)  # 写入文件中
 
    print('records 文件保存完毕.')
    writer.close()

#### 2. 读取TFRecord文件
1. tf.train.string_input_producer 使用参数提供的文件列表创建一个输入队列     
   同时打开多个文件，显示创建Queue，同时隐含了QueueRunner的创建    
1. 生成的队列可以同时被多个文件读取线程操作，输入队列会将文件均匀分给不同线程    
1. 当输入队列文件处理完成后，后将初始化是提供的文件列表中的文件重新加入队列，    
   num_epochs参数限制加载文件列表的最大轮数，当所有文件已经被使用了设定的轮    
   数后，继续读取就会报OutOfRange错误    



In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np


# 读取文件。
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["/home/jovyan/data-vol-1/test/testdata.tfrecords"])
_,serialized_example = reader.read(filename_queue)

# 解析读取的样例。
features = tf.parse_single_example(
    serialized_example,
    features={
        'img_raw':tf.FixedLenFeature([],tf.string),
        'img_width':tf.FixedLenFeature([],tf.int64),
        'img_height':tf.FixedLenFeature([],tf.int64),
        'label':tf.FixedLenFeature([],tf.int64)
    })

images = tf.decode_raw(features['img_raw'],tf.uint8)
labels = tf.cast(features['label'],tf.int32)
img_w = tf.cast(features['img_width'],tf.int32)
img_h = tf.cast(features['img_height'],tf.int32)
sess = tf.Session()

# 启动多线程处理输入数据。
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

for i in range(10):
    image, label, w, h = sess.run([images, labels, img_w, img_h])
    print("i=",i,len(image),label,w,h)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
i= 0 784 7 28 28
i= 1 784 2 28 28
i= 2 784 1 28 28
i= 3 784 0 28 28
i= 4 784 4 28 28
i= 5 784 1 28 28
i= 6 784 4 28 28
i= 7 784 9 28 28
i= 8 784 5 28 28
i= 9 784 9 28 28


In [ ]:

"""
读取二进制文件转换成张量，写进TFRecords,同时读取TFRcords
"""
 
#命令行参数
FLAGS = tf.app.flags.FLAGS       #获取值
tf.app.flags.DEFINE_string("tfrecord_dir","/home/jovyan/data-vol-1/cifar10/cifar10.tfrecords","写入图片数据文件的文件名")
 
 
#读取二进制转换文件
class CifarRead(object):
    """
    读取二进制文件转换成张量，写进TFRecords,同时读取TFRcords
    """
    def __init__(self,file_list):
        """
        初始化图片参数
        :param file_list:图片的路径名称列表
        """
 
        #文件列表
        self.file_list = file_list
 
        #图片大小，二进制文件字节数
        self.height = 32
        self.width = 32
        self.channel = 3
        self.label_bytes = 1
        self.image_bytes = self.height * self.width * self.channel
        self.bytes = self.label_bytes + self.image_bytes
 
 
    def read_and_decode(self):
        """
        解析二进制文件到张量
        :return: 批处理的image,label张量
        """
        #1.构造文件队列
        file_queue = tf.train.string_input_producer(self.file_list)
 
        #2.阅读器读取内容
        reader = tf.FixedLengthRecordReader(self.bytes)
 
        key ,value = reader.read(file_queue)    #key为文件名，value为元组
 
        print("reader.read:",type(key),type(value))
 
        #3.进行解码，处理格式
        label_image = tf.decode_raw(value,tf.uint8)
        print(label_image)
 
        #处理格式，image，label
        #进行切片处理，标签值
        #tf.cast()函数是转换数据格式，此处是将label二进制数据转换成int32格式
        label = tf.cast(tf.slice(label_image,[0],[self.label_bytes]),tf.int32)
 
        #处理图片数据
        image = tf.slice(label_image,[self.label_bytes],[self.image_bytes])
        print(image)
 
        #处理图片的形状，提供给批处理
        #因为image的形状已经固定，此处形状用动态形状来改变
        image_tensor = tf.reshape(image,[self.height,self.width,self.channel])
        print(image_tensor)
 
        #批处理图片数据
        image_batch,label_batch = tf.train.batch([image_tensor,label],batch_size=10,num_threads=1,capacity=10)
 
        return image_batch,label_batch
 
    def write_to_tfrecords(self,image_batch,label_batch):
        """
        将文件写入到TFRecords文件中
        :param image_batch:
        :param label_batch:
        :return:
        """
 
        #建立TFRecords文件存储器
        writer = tf.python_io.TFRecordWriter(FLAGS.tfrecord_dir)      #传进去命令行参数
 
        #循环取出每个样本的值，构造example协议块
        for i in range(10):
 
            #取出图片的值，  #写进去的是值，而不是tensor类型，
            # 写入example需要bytes文件格式，将tensor转化为bytes用tostring()来转化
            image = image_batch[i].eval().tostring()
 
            #取出标签值，写入example中需要使用int形式，所以需要强制转换int
            label = int(label_batch[i].eval()[0])
 
            #构造每个样本的example协议块
            example = tf.train.Example(features = tf.train.Features(feature = {
                "image":tf.train.Feature(bytes_list = tf.train.BytesList(value = [image])),
                "label":tf.train.Feature(int64_list = tf.train.Int64List(value = [label]))
            }))
 
            #写进去序列化后的值
            writer.write(example.SerializeToString())     #此处其实是将其压缩成一个二进制数据
 
        writer.close()
 
        return None
 
 
 
    def read_from_tfrecords(self):
        """
        从TFRecords文件当中读取图片数据（解析example)
        :param self:
        :return: image_batch,label_batch
        """
 
        #1.构造文件队列
        file_queue = tf.train.string_input_producer([FLAGS.tfrecord_dir])    #参数为文件名列表
 
        #2.构造阅读器
        reader = tf.TFRecordReader()
 
        key,value = reader.read(file_queue)
 
        #3.解析协议块,返回的值是字典
        feature = tf.parse_single_example(value,features={
            "image":tf.FixedLenFeature([],tf.string),
            "label":tf.FixedLenFeature([],tf.int64)
        })
 
        #feature["image"],feature["label"]
        #处理标签数据    ，cast()只能在int和float之间进行转换
        label = tf.cast(feature["label"],tf.int32)    #将数据类型int64 转换为int32
 
        #处理图片数据，由于是一个string,要进行解码，  #将字节转换为数字向量表示，字节为一字符串类型的张量
        #如果之前用了tostring(),那么必须要用decode_raw()转换为最初的int类型
        # decode_raw()可以将数据从string,bytes转换为int，float类型的
        image = tf.decode_raw(feature["image"],tf.uint8)
 
        #转换图片的形状，此处需要用动态形状进行转换
        image_tensor = tf.reshape(image,[self.height,self.width,self.channel])
 
        #4.批处理
        image_batch,label_batch = tf.train.batch([image_tensor,label],batch_size=10,num_threads=1,capacity=10)
 
        return image_batch,label_batch
 
 
if __name__ == '__main__':
 
    # 找到文件路径，名字，构造路径+文件名的列表,"A.csv"...
    # os.listdir() 方法用于返回指定的文件夹包含的文件或文件夹的名字的列表
    filename = os.listdir('/home/jovyan/data-vol-1/cifar10/cifar-10-batches-bin/')
 
    #加上路径
    file_list = [os.path.join('/home/jovyan/data-vol-1/cifar10/cifar-10-batches-bin/', file) for file in filename if file[-3:] == "bin"]
 
    print("file_list:",file_list)
    #初始化参数
    cr = CifarRead(file_list)
 
    #读取二进制文件
    # image_batch,label_batch = cr.read_and_decode()
 
    #从已经存储的TFRecords文件中解析出原始数据
    image_batch, label_batch = cr.read_from_tfrecords()
 
    with tf.Session() as sess:
        #线程协调器
        coord = tf.train.Coordinator()
 
        #开启线程
        threads = tf.train.start_queue_runners(sess,coord=coord)
 
        print(sess.run([image_batch,label_batch]))
 
        # print("存进TFRecords文件")
        # cr.write_to_tfrecords(image_batch,label_batch)
        # print("存进文件完毕")
 
        #回收线程
        coord.request_stop()
        coord.join(threads)